In the previous notebook, I ended up with a dataframe with all necessary rows and clean columns: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/preprocessing1b_get_usable_df.ipynb

Here, I'll create a column that indicates how many times any given item has been ordered by a user in the past. Then use that to calculate the percentage of past orders where a user has purchased any given item. 

My intuition is that cross-row calculations don't count as leakage and negatively impact modeling IF I'm adding data that has exclusively to do with past orders. If this logic turns out to be inappropriate, I can just come back and split the set into separate users (or into certain orders per user) and re-run any subsequent code. 

Once that row has been completed, I'll engineer columns for product keywords and do any other feature engineering that is possible prior to certain encoding and standardization that is better done as part of a modeling pipeline. 

In [1]:
import pandas as pd
import numpy as np
import os
from library.sb_utils import save_file

import random

from IPython.display import Audio
sound_file = './alert.wav'

In [2]:
df = pd.read_csv('../data/processed/users_all_rows_clean.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218232 entries, 0 to 218231
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                218232 non-null  int64  
 1   user_id                 218232 non-null  int64  
 2   order_by_user_sequence  218232 non-null  int64  
 3   order_dow               218232 non-null  int64  
 4   order_hour_of_day       218232 non-null  int64  
 5   days_since_prior_order  218232 non-null  float64
 6   add_to_cart_sequence    218232 non-null  int64  
 7   reordered               218232 non-null  int64  
 8   product_name            218232 non-null  object 
 9   aisle_name              218232 non-null  object 
 10  dept_name               218232 non-null  object 
dtypes: float64(1), int64(7), object(3)
memory usage: 18.3+ MB


In [3]:
df['user_id'].unique()[0:5]

array([ 275,  960, 1442, 1542, 3035])

In [4]:
# I'd tried a for loop when grouping wasn't working.
# Initiate the column, starting with all items having been ordered 0 times in the past.
#df['past_orders'] = 0
#users = df['user_id'].unique()

#for user in users:
    #user_rows = df.loc[df['user_id']==user, :].copy()
    
    # Iterate over rows starting at 2, because order 1 has 0 items ordered in the past
    #for order in range(2,101):
        
        # Find items that were purchased in the preceding order.
        #purchased_preceding = user_rows[(user_rows['order_by_user_sequence']==(order-1)) 
                                               #& (user_rows['add_to_cart_sequence']>0)][
            #'product_name'].unique()
        
        # Add one to past_orders column for this and all following orders where these 
        # products appear for this user. 
        #user_rows[(user_rows['order_by_user_sequence']>=order)
                   #& (user_rows['product_name'].isin(purchased_preceding))]['past_orders'] +=1
    
    # Replace df values in past_orders column & this user's rows with the new values
    #user_past_orders = user_rows.loc[:,'past_orders'].copy()
    #df = df.merge(user_past_orders)

#Audio(sound_file, autoplay=True)

In [5]:
# In order to group the dataframe and get a cumsum of purchases, I'll need a 'purchased' column

df['purchased'] = 0
df['purchased'] = df[df['add_to_cart_sequence']>0]['purchased'].replace(0,1)
df['purchased'] = df['purchased'].fillna(0)

df['purchased'].value_counts()

0.0    182154
1.0     36078
Name: purchased, dtype: int64

In [6]:
relevant_cols = df.loc[:,['user_id', 'order_by_user_sequence', 'purchased', 'product_name']]
grouped_df = relevant_cols.groupby(['user_id', 'product_name', 
                                    'order_by_user_sequence']).sum().groupby(
    level=1).cumsum().reset_index()
grouped_df.head(15)

,user_id,product_name,order_by_user_sequence,purchased
0,275,Arrowroot Starch/Flour,1,1.0
1,275,Arrowroot Starch/Flour,2,1.0
2,275,Arrowroot Starch/Flour,3,2.0
3,275,Arrowroot Starch/Flour,4,2.0
4,275,Arrowroot Starch/Flour,5,2.0
5,275,Arrowroot Starch/Flour,6,2.0
6,275,Arrowroot Starch/Flour,7,2.0
7,275,Arrowroot Starch/Flour,8,2.0
8,275,Arrowroot Starch/Flour,9,2.0
9,275,Arrowroot Starch/Flour,10,2.0


In [7]:
df[(df['user_id']==275)&(df['product_name']=='Arrowroot Starch/Flour')]

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,purchased
25,1098729,275,1,4,18,-1.0,19,0,Arrowroot Starch/Flour,baking ingredients,pantry,1.0
46,1235105,275,3,5,13,3.0,5,1,Arrowroot Starch/Flour,baking ingredients,pantry,1.0
82936,1631174,275,2,2,15,12.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0
82992,368961,275,4,2,11,25.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0
83027,2694754,275,5,1,12,13.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0
83069,1611411,275,6,5,14,30.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0
83117,32562,275,7,0,17,2.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0
83171,1112478,275,8,3,12,3.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0
83224,3170214,275,9,0,16,11.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0
83283,2893947,275,10,6,20,20.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0


The 'purchased' column in the grouped_df does correctly show how many times an item has been purchased. With user 275, they first purchased arrowroot starch on their first order. Then, the value goes up to 2 when they order it again in order 3, and it remains 2 for all subsequent orders because they don't buy it again. One problem is that the purchased column shows a '1' and a '2' in the rows where they first purchase and reorder the item. To show only how many times they've purchased something in the past, I'll need to subtract 1 from every item in the 'purchased' column where add_to_cart_sequence>0. But first, get values from grouped_df into the full df. 

In [8]:
df = df.merge(grouped_df, on=['user_id', 'product_name', 'order_by_user_sequence'])
df.head(2)

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,purchased_x,purchased_y
0,1098729,275,1,4,18,-1.0,4,0,Organic Half & Half,cream,dairy eggs,1.0,1.0
1,1098729,275,1,4,18,-1.0,2,0,Organic Whole Milk,milk,dairy eggs,1.0,1.0
2,1098729,275,1,4,18,-1.0,1,0,Naturals Chicken Nuggets,frozen appetizers sides,frozen,1.0,1.0
3,1098729,275,1,4,18,-1.0,24,0,Organic Unsalted Butter,butter,dairy eggs,1.0,1.0
4,1098729,275,1,4,18,-1.0,3,0,Organic Unsweetened Almond Milk,soy lactosefree,dairy eggs,1.0,1.0


In [9]:
df[(df['user_id']==275)&(df['product_name']=='Arrowroot Starch/Flour')]

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,purchased_x,purchased_y
25,1098729,275,1,4,18,-1.0,19,0,Arrowroot Starch/Flour,baking ingredients,pantry,1.0,1.0
46,1235105,275,3,5,13,3.0,5,1,Arrowroot Starch/Flour,baking ingredients,pantry,1.0,2.0
82936,1631174,275,2,2,15,12.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,1.0
82992,368961,275,4,2,11,25.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83027,2694754,275,5,1,12,13.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83069,1611411,275,6,5,14,30.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83117,32562,275,7,0,17,2.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83171,1112478,275,8,3,12,3.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83224,3170214,275,9,0,16,11.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83283,2893947,275,10,6,20,20.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0


In [10]:
# The merge succeeded. Work with purchased_y column to make sure it only shows past purchases.
df.loc[df['add_to_cart_sequence']>0, 'purchased_y'] -= 1
df[(df['user_id']==275)&(df['product_name']=='Arrowroot Starch/Flour')]

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,purchased_x,purchased_y
25,1098729,275,1,4,18,-1.0,19,0,Arrowroot Starch/Flour,baking ingredients,pantry,1.0,0.0
46,1235105,275,3,5,13,3.0,5,1,Arrowroot Starch/Flour,baking ingredients,pantry,1.0,1.0
82936,1631174,275,2,2,15,12.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,1.0
82992,368961,275,4,2,11,25.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83027,2694754,275,5,1,12,13.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83069,1611411,275,6,5,14,30.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83117,32562,275,7,0,17,2.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83171,1112478,275,8,3,12,3.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83224,3170214,275,9,0,16,11.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0
83283,2893947,275,10,6,20,20.0,0,0,Arrowroot Starch/Flour,baking ingredients,pantry,0.0,2.0


In [11]:
# The purchases_y column now contains the values I want. Clean this up.
df = df.drop(columns='purchased_x')
df = df.rename(columns={'purchased_y':'prior_purchases'})
df.head()

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name,prior_purchases
0,1098729,275,1,4,18,-1.0,4,0,Organic Half & Half,cream,dairy eggs,0.0
1,1098729,275,1,4,18,-1.0,2,0,Organic Whole Milk,milk,dairy eggs,0.0
2,1098729,275,1,4,18,-1.0,1,0,Naturals Chicken Nuggets,frozen appetizers sides,frozen,0.0
3,1098729,275,1,4,18,-1.0,24,0,Organic Unsalted Butter,butter,dairy eggs,0.0
4,1098729,275,1,4,18,-1.0,3,0,Organic Unsweetened Almond Milk,soy lactosefree,dairy eggs,0.0


In [14]:
# Check to see if this worked. 

In [16]:
# Engineer columns for product keywords

In [17]:
# Change format of dow, hour columns

In [18]:
# Can't do the rest of the normalization until I have a train/test split and pipeline for it
# Save work done so far as new csv file and pick up in next notebook